# Benchmark

We use a list of public datasets to benchmark all the algorithms in StreamAD. Thanks!

1. [AIOps Challenge public dataset for KPI anomaly detection](https://github.com/NetManAIOps/KPI-Anomaly-Detection)
2. 

Firstly, we create a dictionary of the datasets, and download those public datasets.

In [2]:
import os
import subprocess
import pandas as pd
from streamad.util import StreamGenerator, CustomDS, plot
from streamad.model import SpotDetector
from tqdm import tqdm

path = './streamad-benchmark-dataset'
if not os.path.exists(path):
    os.makedirs(path)

# Dataset 1

# Git clone and unzip the files
subprocess.check_call(["git", "clone", "--depth=1", "https://github.com/NetManAIOps/KPI-Anomaly-Detection.git", path +'/dataset1'])
subprocess.check_call(["unzip", "streamad-benchmark-dataset/dataset1/Finals_dataset/phase2_ground_truth.hdf.zip", "-d","streamad-benchmark-dataset/dataset1/Finals_dataset/"])
subprocess.check_call(["unzip", "streamad-benchmark-dataset/dataset1/Finals_dataset/phase2_train.csv.zip", "-d","streamad-benchmark-dataset/dataset1/Finals_dataset/"])

In [9]:
# For preliminary datasets.
df_pre_train = pd.read_csv(path + '/dataset1/Preliminary_dataset/train.csv')

,timestamp,value,label,KPI ID
0,1493568000,1.901639,0,02e99bd4f6cfb33f
1,1493568060,1.786885,0,02e99bd4f6cfb33f
2,1493568120,2.000000,0,02e99bd4f6cfb33f
3,1493568180,1.885246,0,02e99bd4f6cfb33f
4,1493568240,1.819672,0,02e99bd4f6cfb33f
...,...,...,...,...
2476310,1496895300,0.161922,1,88cf3a776ba00e7c
2476311,1496895360,0.162297,1,88cf3a776ba00e7c
2476312,1496895420,0.160597,1,88cf3a776ba00e7c
2476313,1496895480,0.160393,1,88cf3a776ba00e7c


In [34]:
df_pre_train_groups = df_pre_train.groupby('KPI ID')
keys = df_pre_train_groups.groups.keys()

for key in keys:
    df = df_pre_train_groups.get_group(key)
    ds = df[['timestamp', 'value', 'label']]
    label = df['label']

    ds = CustomDS(ds, label)
    stream = StreamGenerator(ds.data)
    model = SpotDetector()

    scores = []

    for x in tqdm(stream.iter_item(),total=len(ds.data)): 
        score = model.fit_score(x)
        scores.append(score)

    data, label, date, features = ds.data, ds.label, ds.date, ds.features
    # plot(data=data, scores=scores, date=date, features=features, label=label)

    break


100%|██████████| 128562/128562 [04:33<00:00, 469.76it/s] 


In [56]:
plot(data=data[:10000], scores=scores[:10000], date=date[:10000], features=features, label=label[:10000])

In [52]:
from streamad.evaluate import (
    NumentaAwareMetircs,
    PointAwareMetircs,
    SeriesAwareMetircs,
)

metric = PointAwareMetircs(anomaly_threshold=0.5)

(precision, recall, f1,) = metric.evaluate(label, tmp_score)
print(precision,recall,f1)


In [8]:
pd.read_hdf("streamad-benchmark-dataset/dataset1/Finals_dataset/phase2_ground_truth.hdf")

,timestamp,value,label,KPI ID
107717,1482940800,0.048066,0,da10a69f-d836-3baa-ad40-3e548ecf1fbd
107718,1482940860,0.010651,0,da10a69f-d836-3baa-ad40-3e548ecf1fbd
107719,1482940920,0.014996,0,da10a69f-d836-3baa-ad40-3e548ecf1fbd
107720,1482940980,0.039722,0,da10a69f-d836-3baa-ad40-3e548ecf1fbd
107721,1482941040,0.022452,0,da10a69f-d836-3baa-ad40-3e548ecf1fbd
...,...,...,...,...
295409,1508984160,34.940000,0,42d6616d-c9c5-370a-a8ba-17ead74f3114
295410,1508984220,35.200000,0,42d6616d-c9c5-370a-a8ba-17ead74f3114
295411,1508984280,33.020000,0,42d6616d-c9c5-370a-a8ba-17ead74f3114
295412,1508984340,35.150000,0,42d6616d-c9c5-370a-a8ba-17ead74f3114
